In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("csvs/airplane_price_dataset.csv")
data = pd.DataFrame(data)
data

,Model,Üretim Yılı,Motor Sayısı,Motor Türü,Kapasite,Menzil (km),Yakıt Tüketimi (L/saat),Saatlik Bakım Maliyeti ($),Yaş,Satış Bölgesi,Fiyat ($)
0,Bombardier CRJ200,1987,2,Turbofan,50,3000,14.36,2185.43,36,Asya,1.285708e+07
1,Bombardier CRJ200,1997,2,Turbofan,50,3000,4.03,1202.08,26,Avrupa,1.391406e+07
2,Airbus A320,1988,2,Turbofan,180,6300,13.26,761.38,35,Avustralya,9.073570e+07
3,Boeing 737,2023,2,Turbofan,162,5700,14.61,592.63,0,Avustralya,1.366597e+08
4,Cessna 172,1985,1,Piston,4,1285,18.49,4245.99,38,Güney Amerika,2.037981e+05
...,...,...,...,...,...,...,...,...,...,...,...
12372,Boeing 777,1994,2,Turbofan,396,15600,12.16,3054.29,29,Avrupa,3.898071e+08
12373,Bombardier CRJ200,2005,2,Turbofan,50,3000,8.26,3107.67,18,Afrika,1.593888e+07
12374,Boeing 737,2015,2,Turbofan,162,5700,6.93,3972.16,8,Kuzey Amerika,9.870994e+07
12375,Cessna 172,2013,1,Piston,4,1285,14.28,999.84,10,Afrika,3.965207e+05


In [3]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeRegressor as random
num_cols = data.select_dtypes(include=[int]).columns
obj_cols = data.select_dtypes(include=[object]).columns
nums_trans = SimpleImputer(strategy='mean')
obj_oh_trans = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])
bundle = ColumnTransformer(transformers=[
    ("nums" , nums_trans, num_cols),
    ("obj_oh", obj_oh_trans, obj_cols)
])
model = random(random_state=0)
my_pipeline = Pipeline(steps=[
    ('preprocessor', bundle),
    ('model', model)
])


In [4]:
data.columns

Index(['Model', 'Üretim Yılı', 'Motor Sayısı', 'Motor Türü', 'Kapasite',
       'Menzil (km)', 'Yakıt Tüketimi (L/saat)', 'Saatlik Bakım Maliyeti ($)',
       'Yaş', 'Satış Bölgesi', 'Fiyat ($)'],
      dtype='object')

In [5]:
X = data.drop('Fiyat ($)', axis=1)
y = data['Fiyat ($)']
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)
my_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('nums', SimpleImputer(),
                                                  Index(['Üretim Yılı', 'Motor Sayısı', 'Kapasite', 'Menzil (km)', 'Yaş'], dtype='object')),
                                                 ('obj_oh',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['Model', 'Motor Türü', 'Satış Bölgesi'], dtype='object'))])),
                ('model', DecisionTreeRegressor(random_state=0))])

In [6]:
preds = my_pipeline.predict(X_valid)
from sklearn.metrics import mean_absolute_error
print("MAE:", mean_absolute_error(y_valid, preds))

MAE: 17689025.927562103
